# ONNX ResNet Model

This example will show inference over an exported [ONNX](https://github.com/onnx/onnx) ResNet model using Seldon Core. We will use the Seldon S2I wrapper for Intel's NGraph. The example follows this [NGraph tutorial](https://ai.intel.com/adaptable-deep-learning-solutions-with-ngraph-compiler-and-onnx/).

 Preequisites:
   * torch==0.4.0
   * torchvision==0.2.1
   * protoc > 3.4.0

In [ ]:
%matplotlib inline
from torch.autograd import Variable
import torch.onnx
import torchvision
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image
import ngraph as ng
import numpy as np
import matplotlib.pyplot as plt

Download ResNet pretrained model from Torch and export to ONNX file.

In [ ]:
# ImageNet input has 3 channels and 224x224 resolution
imagenet_input = Variable(torch.randn(1, 3, 224, 224))

# Download ResNet (or construct your model)
model = torchvision.models.resnet50(pretrained=True)

# Export model to an ONNX file
torch.onnx.export(model, imagenet_input, 'resnet.onnx')

## Test Model
Load ONNX model into ngraph.

In [ ]:
from ngraph_onnx.onnx_importer.importer import import_onnx_file

# Import the ONNX file
models = import_onnx_file('resnet.onnx')

# Create an nGraph runtime environment
runtime = ng.runtime(backend_name='CPU')

# Select the first model and compile it to a callable function
model = models[0]
resnet = runtime.computation(model['output'], *model['inputs'])


Test on an image of a Zebra.

In [ ]:
img = image.load_img('zebra.jpg', target_size=(224, 224))
img = image.img_to_array(img)
plt.imshow(img / 255.)
x = np.expand_dims(img.copy(), axis=0)
x = preprocess_input(x,mode='torch')
x = x.transpose(0,3,1,2)
preds = resnet(x)
decode_predictions(preds, top=5)

## Package Model Using S2I

In [ ]:
!s2i build . seldonio/seldon-core-s2i-python3-ngraph-onnx:0.2 onnx-resnet:0.1

In [ ]:
!docker run --name "onnx_resnet_predictor" -d --rm -p 5000:5000 onnx-resnet:0.1

In [ ]:
!cd ../../../wrappers/testing && make build_protos

In [ ]:
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p

In [ ]:
!docker rm onnx_resnet_predictor --force

# Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need Minikube version 0.25.2**

In [ ]:
!minikube start --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

In [ ]:
!helm init

In [ ]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

In [ ]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../helm-charts/seldon-core --name seldon-core

In [ ]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver

In [ ]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-python3-ngraph-onnx:0.2 onnx-resnet:0.1

In [ ]:
!kubectl create -f onnx_resnet_deployment.json

Wait until deployment is running

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

In [ ]:
!cd ../../../util/api_tester && make build_protos

In [ ]:
!python ../../../util/api_tester/api-tester.py contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

In [ ]:
!minikube delete
